In [8]:
import torch
import random
import zipfile
device= torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.__version__)
print(device)

1.8.0
cuda


In [9]:
with zipfile.ZipFile("../../data/jaychou_lyrics.txt.zip") as zin:#读取zip文件
    with zin.open('jaychou_lyrics.txt')as f:
        corpus_chars = f.read().decode()
corpus_chars[:40]        
    

'想要有直升机\n想要和你飞到宇宙去\n想要和你融化在一起\n融化在宇宙里\n我每天每天每'

In [10]:
corpus_chars=corpus_chars.replace("\n"," ").replace("\r","")#去掉\n,\r
corpus_chars=corpus_chars[0:10000]


建立索引

In [11]:
idx_to_char=list(set(corpus_chars))
char_to_idx=dict([(char,i)for i,char in enumerate(idx_to_char)])
vocab_size=len(char_to_idx)


In [13]:
corpus_indices=[char_to_idx[char] for char in corpus_chars]
sample=corpus_indices[0:20]
print("文字序列:",''.join([idx_to_char[idx] for idx in sample]))
print("标号序列:",corpus_indices[0:20])

文字序列: 想要有直升机 想要和你飞到宇宙去 想要和
标号序列: [42, 743, 270, 37, 879, 336, 996, 42, 743, 774, 601, 921, 607, 561, 867, 387, 996, 42, 743, 774]


In [14]:
def data_iter_random(corpus_indices,batch_size,num_steps,device=None):
    num_examples=(len(corpus_indices)-1)//num_steps
    epoch_size=num_examples//batch_size
    example_indices=list(range(num_examples))
    random.shuffle(example_indices)
    
    def _data(pos):
        return corpus_indices[pos:pos+num_steps]
    if device is None:
        device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    for i in range(epoch_size):
        i=i*batch_size
        batch_indices=example_indices[i:i+batch_size]
        X = [_data(j * num_steps) for j in batch_indices]
        Y = [_data(j * num_steps + 1) for j in batch_indices]
        yield torch.tensor(X, dtype=torch.float32, device=device), torch.tensor(Y, dtype=torch.float32, device=device)

随机采样

In [15]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def data_iter_random(corpus_indices, batch_size, num_steps, device=None):
    # 减1是因为输出的索引x是相应输入的索引y加1
    num_examples = (len(corpus_indices) - 1) // num_steps
    epoch_size = num_examples // batch_size
    example_indices = list(range(num_examples))
    random.shuffle(example_indices)

    # 返回从pos开始的长为num_steps的序列
    def _data(pos):
        return corpus_indices[pos: pos + num_steps]
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    for i in range(epoch_size):
        # 每次读取batch_size个随机样本
        i = i * batch_size
        batch_indices = example_indices[i: i + batch_size]
        X = [_data(j * num_steps) for j in batch_indices]
        Y = [_data(j * num_steps + 1) for j in batch_indices]
        yield torch.tensor(X, dtype=torch.float32, device=device), torch.tensor(Y, dtype=torch.float32, device=device)

In [16]:
my_seq = list(range(30))
for X, Y in data_iter_random(my_seq, batch_size=2, num_steps=6):
    print('X: ', X, '\nY:', Y, '\n')

X:  tensor([[18., 19., 20., 21., 22., 23.],
        [12., 13., 14., 15., 16., 17.]], device='cuda:0') 
Y: tensor([[19., 20., 21., 22., 23., 24.],
        [13., 14., 15., 16., 17., 18.]], device='cuda:0') 

X:  tensor([[ 6.,  7.,  8.,  9., 10., 11.],
        [ 0.,  1.,  2.,  3.,  4.,  5.]], device='cuda:0') 
Y: tensor([[ 7.,  8.,  9., 10., 11., 12.],
        [ 1.,  2.,  3.,  4.,  5.,  6.]], device='cuda:0') 

